# Driving a Financial Valuation from a Design Scenario

```{warning}
The following is a work in progress. More content will be added as produced.
```

As seen in
[Loading a Design Scenario](../load_design_scenario.ipynb),
a design scenario's object model (a
[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph) including all
its spatial and semantic relationships) can be loaded via Speckle from any of
Speckle's supported [connectors](https://speckle.guide/user/connectors.html), so
long as those objects (`Entity`s and `Relationship`s) have been assigned.

In this notebook, we will demonstrate how to use the object model (graph) to
drive a financial valuation of the design scenario. This is done by calculating
cash flows per `Entity`, and then using the `Relationship`s to aggregate those
`Flow`s into appropriate `Stream`s.

## Load the Design Scenario
We will use the same design scenario as in the previous notebook.

In [1]:
import os

import networkx as nx
import locale
import pandas as pd
from IPython.display import IFrame
import plotly.io as pio
import plotly.express as px
import plotly.offline as py
import pprint as pp
import rangekeeper as rk

In [2]:
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN'))
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)


 SpeckleClient( server: https://speckle.xyz, authenticated: True )


### Inspect the Model Graph

In [3]:
model = speckle.get_commit(stream_id=stream_id)
parsed = rk.api.Speckle.parse(base=model['@scenario'])
scenario = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='design_scenario',
    type='scenario')

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


In [4]:
scenario.plot(name='assets/design_scenario')
IFrame(src="./design_scenario.html", width='100%', height=800)

assets/design_scenario.html


## Assigning and Aggregating `Flow`s per `Entity`
Now we can organise the two main categories of cash flows (costs and revenues)
according to the `Relationship` types in the design scenario.

In this example, revenue-based cash flows are generated by `Entity`s that have a
measureable floor area -- in this case, anything that has a 'gfa' property.

We can query the graph to find them:

### Revenue-producing Entities
Revenue-producing `Entity`s will be aggregated by their spatial containment. So
we can first slice our graph to a tree of nodes that have "contains"
relationships:

In [5]:
spatial_containment = rk.graph.Assembly.from_graph(
    graph=scenario.graph.edge_subgraph(
        [edge for edge in scenario.graph.edges(keys=True) if edge[2] == 'spatiallyContains']),
    name='spatial_containment',
    type='subgraph')

Note: we anticipate that the spatial decomposition is completely hierarchical
(i.e., no spaces overlap, which means no spaces have multiple parents). We can
check this by testing whether the containment graph is a "tree" (or
"arborescence": see [NetworkX Tree](https://networkx.org/documentation/stable/reference/algorithms/tree.html#tree)).

In [6]:
nx.is_arborescence(spatial_containment.graph)

True

In [7]:
# Plot the spatial containment graph
spatial_containment.plot(name='assets/spatial_containment')
IFrame(src="./spatial_containment.html", width='100%', height=800)

assets/spatial_containment.html


#### Simple Aggregation
Now that we have verified this, we can continue with a simple aggregation. We
wish to aggregate up through our hierarchy all `Entity`s that have a Gross Floor
Area ('gfa') property. *Rangekeeper* provides defaults for the `Entity.aggregate()`
method for this, whereby a named property of any `Entity` is summed (numerical
addition) into its parent `Entity` against a specified label:

In [8]:
spatial_containment.aggregate(
    property='gfa',
    label='subtotal_gfa')

We can tabulate this by converting the graph to a pandas `DataFrame`. Note that
the graph nodes (as witnessed in the DataFrame's index and 'parent' column)
are GUID strings. Each of these have corresponding `Entity` objects, organised
via NetworkX's Node Attributes. See [NetworkX Tutorial](https://networkx.org/documentation/stable/tutorial.html#what-to-use-as-nodes-and-edges)

In [9]:
df = spatial_containment.to_DataFrame()
df = df[['name', 'type', 'gfa', 'subtotal_gfa', 'parent', 'use', 'ffl', 'number']]
df

,name,type,gfa,subtotal_gfa,parent,use,ffl,number
003c1ee8-4450-4fb1-98bd-54d11408d99e,development,development,NaN,45816.419959,None,NaN,NaN,NaN
eb42cd14-73ab-41fc-99c0-c46631ee1208,buildingA,building,NaN,19605.790919,003c1ee8-4450-4fb1-98bd-54d11408d99e,NaN,NaN,NaN
dac040a5-2605-4cd0-a2fb-40ebb85450d9,plinth,building,NaN,12773.744211,003c1ee8-4450-4fb1-98bd-54d11408d99e,NaN,NaN,NaN
8c7c9be7-fd8f-4527-ae90-24e15c3581db,buildingB,building,NaN,13436.884829,003c1ee8-4450-4fb1-98bd-54d11408d99e,NaN,NaN,NaN
f7e2fa84-54a6-4fd4-ae93-f339947035b6,utilities,utilities,NaN,0.000000,003c1ee8-4450-4fb1-98bd-54d11408d99e,NaN,NaN,NaN
655b2dfe-5c18-4d68-95f0-f5d27804430d,buildingAoffice,space,NaN,14691.140177,eb42cd14-73ab-41fc-99c0-c46631ee1208,office,NaN,NaN
47dfa8b6-e213-4099-9f5b-834df0ca34b0,buildingAretail,space,NaN,1123.715326,eb42cd14-73ab-41fc-99c0-c46631ee1208,retail,NaN,NaN
da796c3e-94b0-4175-8c6f-ac63e5e60714,buildingAresidential,space,NaN,2296.143091,eb42cd14-73ab-41fc-99c0-c46631ee1208,residential,NaN,NaN
08e12c72-4450-424f-8ff4-a4da7246ee05,buildingAretail,space,NaN,534.530238,eb42cd14-73ab-41fc-99c0-c46631ee1208,retail,NaN,NaN
634c5fb4-f204-4287-8c80-f23371056f1c,buildingAparking,space,NaN,960.262087,eb42cd14-73ab-41fc-99c0-c46631ee1208,parking,NaN,NaN


We can also plot this as a hierarchical pie chart (a.k.a. 'sunburst' chart):

In [10]:
fig = spatial_containment.sunburst(property='subtotal_gfa')
filename='spatial_containment_sunburst.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800)

#### Aggregating a (Financial) Calculation
A more complex aggregation involves collecting specific cash `Flow`s into
`Stream`s per `Entity`/`Assembly`; for instance, we may want to aggregate all
revenues generated by lettable floorspace into their respective parent `Entity`s,
in order to analyse or compare the performance of the scenario at different
resolutions.

We start by identifying which `Entity`s are 'floor' types and have an area
measurement:

In [11]:
floors = [entity for (entityId, entity) in spatial_containment.get_entities().items() if entity['type'] == 'floor' and hasattr(entity, 'gfa')]

For each floor we can project a set of simple cash flows, in a similar manner
to [A Basic DCF Valuation](../basic_dcf.ipynb).

First initialize our locale, and a set of parameters, including some around
area efficiency ratios, initial incomes per area, and growth rates:

In [12]:
locale.setlocale(locale.LC_ALL, 'en_au')
units = rk.measure.Index.registry
currency = rk.measure.register_currency(registry=units)
period_type = rk.periodicity.Type.YEAR

In [13]:
efficiency_ratios = {
    'office': 0.88,
    'retail': 0.75,
    'residential': 0.82,
    'parking': 0.93,
    }
initial_income_per_area_pa = {
    'office': 750,
    'retail': 1000,
    'residential': 600,
    'parking': 0,
    }
growth_rates = {
    'office': 0.02,
    'retail': 0.05,
    'residential': 0.035,
    'parking': 0,
    }
vacancy_rates = {
    'office': 0.075,
    'retail': 0.5,
    'residential': 0.02,
    'parking': 0,
    }

In [14]:
inputs = pd.DataFrame(
    data={
        'efficiency_ratio': efficiency_ratios,
        'initial_income_per_area_pa': initial_income_per_area_pa,
        'growth_rate': growth_rates,
        'vacancy_rate': vacancy_rates,
        })
inputs

,efficiency_ratio,initial_income_per_area_pa,growth_rate,vacancy_rate
office,0.88,750,0.020,0.075
retail,0.75,1000,0.050,0.500
residential,0.82,600,0.035,0.020
parking,0.93,0,0.000,0.000


In [15]:
params = {
    'start_date': pd.Timestamp('2001-01-01'),
    'num_periods': 10,
    'period_type': rk.periodicity.Type.YEAR,
    }

##### `Stream`s of Revenue `Flow`s per `Entity`:
It is now simple to construct a set of cash flows per `Entity` in our design
scenario. We will use a similar structure to the previous notebooks.

First, we will define a `Revenues` class that holds the parameters:

In [16]:
class Revenues:
    def __init__(self, params: dict):
        self.params = params

Next we add our `Span` information:

In [17]:
@rk.update_class(Revenues)
class Revenues:
    def init_spans(self):
        self.calc_span = rk.span.Span.from_num_periods(
            name='Span to Calculate Reversion',
            date=self.params['start_date'],
            period_type=self.params['period_type'],
            num_periods=self.params['num_periods'] + 1)
        self.acq_span = rk.span.Span.from_num_periods(
            name='Acquisition Span',
            date=rk.periodicity.offset_date(
                self.params['start_date'],
                num_periods=-1,
                period_type=self.params['period_type']),
            period_type=self.params['period_type'],
            num_periods=1)
        self.span = self.calc_span.shift(
            name='Span',
            num_periods=-1,
            period_type=self.params['period_type'],
            bound='end')

And then we add Revenue `Flow`s:

In [18]:
@rk.update_class(Revenues)
class Revenues:
    def init_flows(self):
        self.pgi = rk.flux.Flow.from_projection(
            name='Potential Gross Income',
            value=self.params['initial_income'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['growth_rate']),
            sequence=self.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        self.vacancy = rk.flux.Flow(
            name='Vacancy Allowance',
            movements=self.pgi.movements * -self.params['vacancy_rate'],
            units=currency.units)
        self.egi = rk.flux.Stream(
            name='Effective Gross Income',
            flows=[self.pgi, self.vacancy],
            period_type=self.params['period_type'])

Now we need to add `Entity`-specific parameters to the parameters dictionary,
and run  the `init_spans()` and `init_flows()` methods per floor `Entity`:

(Note we place the resultant `Stream` in the `events` attribute of the `Entity`)

In [19]:
for floor in floors:
    parent = floor.get_relatives(
        relationship_type='spatiallyContains',
        outgoing=False,
        assembly=spatial_containment)[0]
    floor['use'] = parent['use']

    floor.params = params.copy()
    floor.params['initial_income'] = initial_income_per_area_pa[floor['use']] * floor['gfa']
    floor.params['growth_rate'] = growth_rates[floor['use']]
    floor.params['vacancy_rate'] = vacancy_rates[floor['use']]

    floor.events = {}

    revenues = Revenues(floor.params)
    revenues.init_spans()
    revenues.init_flows()
    floor.events = {
        'egi': revenues.egi,
        }

##### Aggregating by Container
Since we have the "containment" hierarchy, we can aggregate the `Revenues` per
parent `Entity`:

*Rangekeeper* provides some helper methods to assist in the aggregation of
`Stream`s and `Flow`s; specifically, since the `Entity.aggregate()` method
updates a specified `Entity`'s property with a dictionary of subentity
properties, keyed by their respective `entityId`s, we can use the
`aggregate_events()` function to support the collation of child `Entity`
`Flow`s into a single `Stream` per label in the `Entity`'s `events` property:

In [20]:
def aggregate_egis(**kwargs):
    return rk.graph.Entity.aggregate_events(
        **kwargs,
        property='egi',
        period_type=period_type)

This function is passed into the `Entity.aggregate()` method as the `function`
parameter:

In [21]:
aggregation = spatial_containment.aggregate(
    function=aggregate_egis,
    property='events',
    label='aggregate_egi',
    relationship_type='spatiallyContains')

Now we can look at what the `aggregate` method has produced for a single floor:

In [22]:
floors[1]['events']['egi']

date,Potential Gross Income,Vacancy Allowance
2001,"$1,086,691.51","-$81,501.86"
2002,"$1,108,425.34","-$83,131.90"
2003,"$1,130,593.84","-$84,794.54"
2004,"$1,153,205.72","-$86,490.43"
2005,"$1,176,269.83","-$88,220.24"
2006,"$1,199,795.23","-$89,984.64"
2007,"$1,223,791.14","-$91,784.34"
2008,"$1,248,266.96","-$93,620.02"
2009,"$1,273,232.30","-$95,492.42"
2010,"$1,298,696.94","-$97,402.27"


We can then inspect the `aggregate_egi` property of an `Entity`, which holds a
`Stream` of `Flow`s (each coming from a child of that `Entity`). Let's use the
office compartment in Building A:

In [23]:
buildingAoffice = [e for (id, e) in spatial_containment.get_entities().items() if e.name == 'buildingAoffice'][0]
print(buildingAoffice.name)
buildingAoffice['aggregate_egi']

buildingAoffice


date,Effective Gross Income for 6bfd12d7-72d7-417f-bc31-1bef4ffaadb0,Effective Gross Income for f25ef6dc-c9cc-444a-b758-e30e87a9593f,Effective Gross Income for 97e6b3b6-edfb-49a9-91f3-337b0b150c21,Effective Gross Income for 6ee48fbb-decc-47ea-8e20-e415c669143f,Effective Gross Income for e57ea01c-ae5d-4281-8aff-56e95d40a239,Effective Gross Income for 59adf9e1-f47d-4bcc-b506-b3a2731654b9,Effective Gross Income for 68f28de9-539f-4276-8dcc-cf4eb7dbe82c,Effective Gross Income for 508b3a7f-7094-4f47-9b61-57ef8f4a6205,Effective Gross Income for 1e6b1a6a-d64a-4c68-9210-3c4f0585000f,Effective Gross Income for 12c94d26-7779-4384-856a-2139664d55e1
2001,"$1,145,271.71","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64"
2002,"$1,168,177.14","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44"
2003,"$1,191,540.69","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30"
2004,"$1,215,371.50","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29"
2005,"$1,239,678.93","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60"
2006,"$1,264,472.51","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59"
2007,"$1,289,761.96","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80"
2008,"$1,315,557.20","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94"
2009,"$1,341,868.34","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88"
2010,"$1,368,705.71","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67"


We can also easily sum and collapse that aggregation, as it is a Stream`:

In [24]:
buildingAoffice['aggregate_egi'].sum()

date,egi for 655b2dfe-5c18-4d68-95f0-f5d27804430d Aggregation (sum)
2001-12-31 00:00:00,"$10,191,978.50"
2002-12-31 00:00:00,"$10,395,818.07"
2003-12-31 00:00:00,"$10,603,734.43"
2004-12-31 00:00:00,"$10,815,809.12"
2005-12-31 00:00:00,"$11,032,125.30"
2006-12-31 00:00:00,"$11,252,767.81"
2007-12-31 00:00:00,"$11,477,823.16"
2008-12-31 00:00:00,"$11,707,379.63"
2009-12-31 00:00:00,"$11,941,527.22"
2010-12-31 00:00:00,"$12,180,357.76"


In [25]:
buildingAoffice['aggregate_egi'].sum().collapse()

date,egi for 655b2dfe-5c18-4d68-95f0-f5d27804430d Aggregation (sum)
2011-12-31 00:00:00,"$124,023,285.90"


THe above subtotals the revenue from the Office space of Building A over the
period of the scenario.

We can then do this for all `Entity`s in the graph, and plot the results:

In [26]:
for (entityId, entity) in spatial_containment.get_entities().items():
    entity['aggregate_egi_value'] = entity['aggregate_egi'].sum().collapse().movements.to_numpy()[0] if entity['aggregate_egi'] is not None else None

In [27]:
fig = spatial_containment.sunburst(property='aggregate_egi_value')
filename='aggregate_egis_sunburst.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800)

In [28]:
# df = px.data.tips()

In [29]:
# fig = px.sunburst(df, path=['day', 'time', 'sex'], values='total_bill')
# fig.show()